In [26]:
import numpy as np
import pandas as pd
pd.set_option("display.precision", 4)

## 1.Zadanie
### Vstupne udaje

In [27]:
# Section properties
E = 2.83*(10**7) #kPa
A = 0.06 #m2
I = 4.5*(10**(-4)) #m-4

# lengths
a = 3 #m
b = 3 #m
c = 3 #m

# loads
F = 40 #kN
q = 5 #kN/m

In [28]:
class MemberStiffnessMatrix:

    def localMember(len, A, E, I):
        return pd.DataFrame([[E*A/len, 0, 0], 
                            [0, 12*E*I/len**3, 6*E*I/len**2], 
                            [0, 6*E*I/len**2, 4*E*I/len]])

    def globalMember(rotationMatrix, localMemberStiffnessMatrix, transformationMatrix):
        A = rotationMatrix
        k = localMemberStiffnessMatrix
        B = transformationMatrix
        
        # shape of final output 
        # [kaa, kab],
        # [kba, kbb]
        
        kaa = A.T*A
        kba = B*kaa
        kab = kba.T
        kbb = B*kab
        
        col1 = pd.concat([kaa, kba])
        col2 = pd.concat([kab,kbb])
        
        return pd.concat([col1, col2], axis=1)

In [29]:
class TransformationMatrices:
    def rotation(alfa):
        alfa_rad = np.deg2rad(alfa)
        return pd.DataFrame([[np.cos(alfa_rad), np.sin(alfa_rad), 0], 
                            [-np.sin(alfa_rad), np.cos(alfa_rad), 0], 
                            [0, 0, 1]])

    def transmission(lx, ly):
        return pd.DataFrame([[-1, 0, 0], 
                            [0, -1, 0], 
                            [-ly, lx, -1]])

In [30]:

class BendingMoment:
    def _init_(self, length, F, q):
        self.length = length
        self.F = F
        self.q = q
        
    def pointLoad(length, F):
        return F*length/8
    
    def distributedLoad(length, q):
        return q*length**2/12
    
class Reactions:
    def _init_(self, length, F, q):
        self.length = length
        self.F = F
        self.q = q
        
    def pointLoad(F):
        return F/2
    
    def distributedLoad(length, q):
        return q*length/2
    
class LoadMatrix:
    def localMatrix(N,V,M):
        return pd.DataFrame([[N], 
                             [V], 
                             [M]])

    def globalMatrix(localMatrix, A0_transp):
        return localMatrix*A0_transp

    def superMatrix(globalMatrix_1, globalMatrix_2):
        return pd.concat([globalMatrix_1, globalMatrix_2]).drop(columns=[1, 2])

In [31]:
%%HTML
<img src="schema.png" width="400" height="400" style="position: relative"/>

### Prut 1-2

In [32]:
k01 = MemberStiffnessMatrix.localMember(c,A,E,I)
k01

,0,1,2
0,566000.0,0.0,0.0
1,0.0,5660.0,8490.0
2,0.0,8490.0,16980.0


In [33]:
%%HTML
<img src="rotation-matrix.png" width="400" height="300"/>

In [34]:
alfa = 270 #deg
A0 = TransformationMatrices.rotation(alfa)
A0

,0,1,2
0,-1.8370e-16,-1.0000e+00,0
1,1.0000e+00,-1.8370e-16,0
2,0.0000e+00,0.0000e+00,1


In [35]:
lx = 0 #m
ly = -3 #m

B = TransformationMatrices.transmission (lx,ly)
B

,0,1,2
0,-1,0,0
1,0,-1,0
2,3,0,-1


In [36]:
k1 = MemberStiffnessMatrix.globalMember(A0, k01, B)
k1

,0,1,2,0,1,2
0,3.3745e-32,-1.0000e+00,0.0,-3.3745e-32,-0.0000e+00,0.0
1,-1.0000e+00,3.3745e-32,0.0,-0.0000e+00,-3.3745e-32,0.0
2,0.0000e+00,0.0000e+00,1.0,0.0000e+00,0.0000e+00,-1.0
0,-3.3745e-32,-0.0000e+00,0.0,3.3745e-32,-0.0000e+00,0.0
1,-0.0000e+00,-3.3745e-32,0.0,-0.0000e+00,3.3745e-32,0.0
2,0.0000e+00,0.0000e+00,-1.0,0.0000e+00,0.0000e+00,1.0


Local loading matrix

F = (N,V,M)

In [37]:
N = 0 #kN
V = Reactions.distributedLoad(c, q) #kN
M = BendingMoment.distributedLoad(c, q) #kNm


In [38]:
F1_l = LoadMatrix.localMatrix(N,V,M)
F2_l = LoadMatrix.localMatrix(N,-V,M)
F2_l

,0
0,0.00
1,-7.50
2,3.75


Global loading matrix

In [39]:
F1_g = LoadMatrix.globalMatrix(F1_l, A0.T)
F2_g = LoadMatrix.globalMatrix(F2_l, A0.T)

LoadMatrix.superMatrix(F1_g, F2_g)

,0
0,-0.0
1,-7.5
2,0.0
0,-0.0
1,7.5
2,0.0


### Prut 2-3

In [40]:
k02 = MemberStiffnessMatrix.localMember(a,A,E,I)
A0 = TransformationMatrices.rotation(alfa = 0)
B = TransformationMatrices.transmission (lx = 3,ly = 0)
k2 = MemberStiffnessMatrix.globalMember(A0, k02, B)
k2


,0,1,2,0,1,2
0,1.0,-0.0,0.0,-1.0,-0.0,0.0
1,-0.0,1.0,0.0,-0.0,-1.0,0.0
2,0.0,0.0,1.0,0.0,0.0,-1.0
0,-1.0,-0.0,0.0,1.0,-0.0,0.0
1,-0.0,-1.0,0.0,-0.0,1.0,0.0
2,0.0,0.0,-1.0,0.0,0.0,1.0


In [41]:
N = 0 #kN
V = Reactions.pointLoad(F) #kN
M = BendingMoment.pointLoad(a,F) #kNm

F1_l = LoadMatrix.localMatrix(N,V,M)
F2_l = LoadMatrix.localMatrix(N,-V,M)

F1_g = LoadMatrix.globalMatrix(F1_l, A0.T)
F2_g = LoadMatrix.globalMatrix(F2_l, A0.T)

LoadMatrix.superMatrix(F1_g, F2_g)

,0
0,0.0
1,0.0
2,0.0
0,0.0
1,-0.0
2,0.0


### Prut 3-4

In [42]:
k03 = MemberStiffnessMatrix.localMember(b,A,E,I)
A0 = TransformationMatrices.rotation(alfa = 90)
B = TransformationMatrices.transmission (lx = 0,ly = -3)
k3 = MemberStiffnessMatrix.globalMember(A0, k03, B)
k3

,0,1,2,0,1,2
0,3.7494e-33,-1.0000e+00,0.0,-3.7494e-33,-0.0000e+00,0.0
1,-1.0000e+00,3.7494e-33,0.0,-0.0000e+00,-3.7494e-33,0.0
2,0.0000e+00,0.0000e+00,1.0,0.0000e+00,0.0000e+00,-1.0
0,-3.7494e-33,-0.0000e+00,0.0,3.7494e-33,-0.0000e+00,0.0
1,-0.0000e+00,-3.7494e-33,0.0,-0.0000e+00,3.7494e-33,0.0
2,0.0000e+00,0.0000e+00,-1.0,0.0000e+00,0.0000e+00,1.0


In [43]:
N = 0 #kN
V = 0 #kN
M = 0 #kN

F1_l = LoadMatrix.localMatrix(N,V,M)
F2_l = LoadMatrix.localMatrix(N,-V,M)

F1_g = LoadMatrix.globalMatrix(F1_l, A0.T)
F2_g = LoadMatrix.globalMatrix(F2_l, A0.T)

LoadMatrix.superMatrix(F1_g, F2_g)

,0
0,0.0
1,0.0
2,0.0
0,0.0
1,0.0
2,0.0


In [45]:
zeros = pd.DataFrame(np.zeros((12, 12)))
zeros

,0,1,2,3,4,5,6,7,8,9,10,11
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
8,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
9,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [46]:
k1.columns=[0,1,2,3,4,5]
k1 = k1.reset_index(drop=True)
k1

,0,1,2,3,4,5
0,3.3745e-32,-1.0000e+00,0.0,-3.3745e-32,-0.0000e+00,0.0
1,-1.0000e+00,3.3745e-32,0.0,-0.0000e+00,-3.3745e-32,0.0
2,0.0000e+00,0.0000e+00,1.0,0.0000e+00,0.0000e+00,-1.0
3,-3.3745e-32,-0.0000e+00,0.0,3.3745e-32,-0.0000e+00,0.0
4,-0.0000e+00,-3.3745e-32,0.0,-0.0000e+00,3.3745e-32,0.0
5,0.0000e+00,0.0000e+00,-1.0,0.0000e+00,0.0000e+00,1.0


In [47]:
layer1 = zeros.add(k1,fill_value=0)
layer1

,0,1,2,3,4,5,6,7,8,9,10,11
0,3.3745e-32,-1.0000e+00,0.0,-3.3745e-32,0.0000e+00,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,-1.0000e+00,3.3745e-32,0.0,0.0000e+00,-3.3745e-32,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0000e+00,0.0000e+00,1.0,0.0000e+00,0.0000e+00,-1.0,0.0,0.0,0.0,0.0,0.0,0.0
3,-3.3745e-32,0.0000e+00,0.0,3.3745e-32,0.0000e+00,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0000e+00,-3.3745e-32,0.0,0.0000e+00,3.3745e-32,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,0.0000e+00,0.0000e+00,-1.0,0.0000e+00,0.0000e+00,1.0,0.0,0.0,0.0,0.0,0.0,0.0
6,0.0000e+00,0.0000e+00,0.0,0.0000e+00,0.0000e+00,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7,0.0000e+00,0.0000e+00,0.0,0.0000e+00,0.0000e+00,0.0,0.0,0.0,0.0,0.0,0.0,0.0
8,0.0000e+00,0.0000e+00,0.0,0.0000e+00,0.0000e+00,0.0,0.0,0.0,0.0,0.0,0.0,0.0
9,0.0000e+00,0.0000e+00,0.0,0.0000e+00,0.0000e+00,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [48]:
k2.columns=[3,4,5,6,7,8]
k2 = k2.reset_index(drop=True).rename(index={0: 3, 1: 4, 2: 5, 3: 6, 4: 7, 5: 8})
k2

,3,4,5,6,7,8
3,1.0,-0.0,0.0,-1.0,-0.0,0.0
4,-0.0,1.0,0.0,-0.0,-1.0,0.0
5,0.0,0.0,1.0,0.0,0.0,-1.0
6,-1.0,-0.0,0.0,1.0,-0.0,0.0
7,-0.0,-1.0,0.0,-0.0,1.0,0.0
8,0.0,0.0,-1.0,0.0,0.0,1.0


In [49]:
layer2 = layer1.add(k2,fill_value=0)
layer2

,0,1,2,3,4,5,6,7,8,9,10,11
0,3.3745e-32,-1.0000e+00,0.0,-3.3745e-32,0.0000e+00,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,-1.0000e+00,3.3745e-32,0.0,0.0000e+00,-3.3745e-32,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0000e+00,0.0000e+00,1.0,0.0000e+00,0.0000e+00,-1.0,0.0,0.0,0.0,0.0,0.0,0.0
3,-3.3745e-32,0.0000e+00,0.0,1.0000e+00,0.0000e+00,0.0,-1.0,0.0,0.0,0.0,0.0,0.0
4,0.0000e+00,-3.3745e-32,0.0,0.0000e+00,1.0000e+00,0.0,0.0,-1.0,0.0,0.0,0.0,0.0
5,0.0000e+00,0.0000e+00,-1.0,0.0000e+00,0.0000e+00,2.0,0.0,0.0,-1.0,0.0,0.0,0.0
6,0.0000e+00,0.0000e+00,0.0,-1.0000e+00,0.0000e+00,0.0,1.0,0.0,0.0,0.0,0.0,0.0
7,0.0000e+00,0.0000e+00,0.0,0.0000e+00,-1.0000e+00,0.0,0.0,1.0,0.0,0.0,0.0,0.0
8,0.0000e+00,0.0000e+00,0.0,0.0000e+00,0.0000e+00,-1.0,0.0,0.0,1.0,0.0,0.0,0.0
9,0.0000e+00,0.0000e+00,0.0,0.0000e+00,0.0000e+00,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [50]:
k3.columns=[6,7,8,9,10,11]
k3 = k3.reset_index(drop=True).rename(index={0: 6, 1: 7, 2: 8, 3: 9, 4: 10, 5: 11})
k3

,6,7,8,9,10,11
6,3.7494e-33,-1.0000e+00,0.0,-3.7494e-33,-0.0000e+00,0.0
7,-1.0000e+00,3.7494e-33,0.0,-0.0000e+00,-3.7494e-33,0.0
8,0.0000e+00,0.0000e+00,1.0,0.0000e+00,0.0000e+00,-1.0
9,-3.7494e-33,-0.0000e+00,0.0,3.7494e-33,-0.0000e+00,0.0
10,-0.0000e+00,-3.7494e-33,0.0,-0.0000e+00,3.7494e-33,0.0
11,0.0000e+00,0.0000e+00,-1.0,0.0000e+00,0.0000e+00,1.0


In [51]:
layer3 = layer2.add(k3,fill_value=0)
layer3

,0,1,2,3,4,5,6,7,8,9,10,11
0,3.3745e-32,-1.0000e+00,0.0,-3.3745e-32,0.0000e+00,0.0,0.0000e+00,0.0000e+00,0.0,0.0000e+00,0.0000e+00,0.0
1,-1.0000e+00,3.3745e-32,0.0,0.0000e+00,-3.3745e-32,0.0,0.0000e+00,0.0000e+00,0.0,0.0000e+00,0.0000e+00,0.0
2,0.0000e+00,0.0000e+00,1.0,0.0000e+00,0.0000e+00,-1.0,0.0000e+00,0.0000e+00,0.0,0.0000e+00,0.0000e+00,0.0
3,-3.3745e-32,0.0000e+00,0.0,1.0000e+00,0.0000e+00,0.0,-1.0000e+00,0.0000e+00,0.0,0.0000e+00,0.0000e+00,0.0
4,0.0000e+00,-3.3745e-32,0.0,0.0000e+00,1.0000e+00,0.0,0.0000e+00,-1.0000e+00,0.0,0.0000e+00,0.0000e+00,0.0
5,0.0000e+00,0.0000e+00,-1.0,0.0000e+00,0.0000e+00,2.0,0.0000e+00,0.0000e+00,-1.0,0.0000e+00,0.0000e+00,0.0
6,0.0000e+00,0.0000e+00,0.0,-1.0000e+00,0.0000e+00,0.0,1.0000e+00,-1.0000e+00,0.0,-3.7494e-33,0.0000e+00,0.0
7,0.0000e+00,0.0000e+00,0.0,0.0000e+00,-1.0000e+00,0.0,-1.0000e+00,1.0000e+00,0.0,0.0000e+00,-3.7494e-33,0.0
8,0.0000e+00,0.0000e+00,0.0,0.0000e+00,0.0000e+00,-1.0,0.0000e+00,0.0000e+00,2.0,0.0000e+00,0.0000e+00,-1.0
9,0.0000e+00,0.0000e+00,0.0,0.0000e+00,0.0000e+00,0.0,-3.7494e-33,0.0000e+00,0.0,3.7494e-33,0.0000e+00,0.0


In [96]:
def highlightNotZero(x):
    if x < 0:
        #red
        color = "#ff9999"
    elif x > 0:
        #green
        color = "#ccffcc"
    else:
        color = "#f2f2f2"
    return f"background: {color}"

def parseZeroValues(x):
    if x != 0:
        return '{:.3e}'
    else:
        return '{:.0f}'


### Final stiffness matrix

In [140]:
layer3.style\
    .format('{:.3e}')\
    .applymap(highlightNotZero)\
    .set_table_styles([{"selector" : "", "props" : [("font-family", "arial")]}])


,0,1,2,3,4,5,6,7,8,9,10,11
0,3.374e-32,-1.000e+00,0.000e+00,-3.374e-32,0.000e+00,0.000e+00,0.000e+00,0.000e+00,0.000e+00,0.000e+00,0.000e+00,0.000e+00
1,-1.000e+00,3.374e-32,0.000e+00,0.000e+00,-3.374e-32,0.000e+00,0.000e+00,0.000e+00,0.000e+00,0.000e+00,0.000e+00,0.000e+00
2,0.000e+00,0.000e+00,1.000e+00,0.000e+00,0.000e+00,-1.000e+00,0.000e+00,0.000e+00,0.000e+00,0.000e+00,0.000e+00,0.000e+00
3,-3.374e-32,0.000e+00,0.000e+00,1.000e+00,0.000e+00,0.000e+00,-1.000e+00,0.000e+00,0.000e+00,0.000e+00,0.000e+00,0.000e+00
4,0.000e+00,-3.374e-32,0.000e+00,0.000e+00,1.000e+00,0.000e+00,0.000e+00,-1.000e+00,0.000e+00,0.000e+00,0.000e+00,0.000e+00
5,0.000e+00,0.000e+00,-1.000e+00,0.000e+00,0.000e+00,2.000e+00,0.000e+00,0.000e+00,-1.000e+00,0.000e+00,0.000e+00,0.000e+00
6,0.000e+00,0.000e+00,0.000e+00,-1.000e+00,0.000e+00,0.000e+00,1.000e+00,-1.000e+00,0.000e+00,-3.749e-33,0.000e+00,0.000e+00
7,0.000e+00,0.000e+00,0.000e+00,0.000e+00,-1.000e+00,0.000e+00,-1.000e+00,1.000e+00,0.000e+00,0.000e+00,-3.749e-33,0.000e+00
8,0.000e+00,0.000e+00,0.000e+00,0.000e+00,0.000e+00,-1.000e+00,0.000e+00,0.000e+00,2.000e+00,0.000e+00,0.000e+00,-1.000e+00
9,0.000e+00,0.000e+00,0.000e+00,0.000e+00,0.000e+00,0.000e+00,-3.749e-33,0.000e+00,0.000e+00,3.749e-33,0.000e+00,0.000e+00
